# Portfolio Optimizer

The portfolio optimizer brings together the power of the Axioma Portfolio Optimizer with Marquee's risk analytics infrastructure
to make minimizing your portfolio's factor risk possible within the same ecosystem.

To use the optimizer, you must have a license to the Axioma Portfolio Optimizer. Please reach out to the
[Marquee sales team](mailto:gs-marquee-sales@ny.email.gs.com?Subject=Portfolio Optimizer Trial Request)
to learn more about how to get a license or how to bring an existing license to Marquee.

## Step 1: Authenticate and Initialize Your Session

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import datetime as dt
import pandas as pd
from IPython.display import display
import warnings
from gs_quant.markets.optimizer import (
    OptimizerUniverse,
    FactorConstraint,
    AssetConstraint,
    SectorConstraint,
    IndustryConstraint,
    OptimizerSettings,
    OptimizerStrategy,
    OptimizerConstraints,
    OptimizerObjective,
    OptimizerType,
    OptimizerObjectiveTerm,
    OptimizerObjectiveParameters,
    ConstraintPriorities,
    PrioritySetting,
    CountryConstraint,
    HedgeTarget,
)
from gs_quant.markets.position_set import Position, PositionSet
from gs_quant.markets.securities import Asset, AssetIdentifier
from gs_quant.models.risk_model import FactorRiskModel
from gs_quant.session import GsSession, Environment
from gs_quant.target.hedge import CorporateActionsTypes
from gs_quant.markets.factor_analytics import FactorAnalytics

client = None
secret = None

## External users must fill in their client ID and secret below and comment out the line below

# client = 'ENTER CLIENT ID'
# secret = 'ENTER CLIENT SECRET'

GsSession.use(Environment.PROD, client_id=client, client_secret=secret)
warnings.filterwarnings("ignore", category=RuntimeWarning)

print('GS Session initialized.')

## Step 2: Define Your Initial Position Set

Use the `PositionSet` class in GS Quant to define the initial holdings to optimize.
You can define your positions as a list of identifiers with quantities or, alternatively, as a
list of identifiers with weights, along with a reference notional value.

*GS Quant will resolve all identifiers (Bloomberg IDs, SEDOLs, RICs, etc) historically as of the optimization date.*

In [ ]:
position_set = PositionSet(
    date=dt.date(day=23, month=4, year=2024),
    reference_notional=10_000_000,
    positions=[Position(identifier='AAPL UW', weight=0.4), Position(identifier='GS UN', weight=0.6)],
)

position_set.resolve()

### Step 2.1: Analyze Initial Portfolio Risk Exposures

Analyze the risk exposures of the initial portfolio using the Marquee PreTrade.

In [ ]:
# Set parameters for factor analysis in PreTrade
risk_model_id = 'AXIOMA_AXUS4S'
currency = 'USD'
participation_rate = 0.1

analytics = FactorAnalytics(risk_model_id=risk_model_id, currency=currency, participation_rate=participation_rate)

print('Running risk analysis on initial portfolio...')
try:
    initial_analysis = analytics.get_factor_analysis(position_set)

    print('\nInitial Portfolio Summary:')
    summary_table = analytics.create_exposure_summary_table(initial_analysis)
    display(summary_table)
except Exception as e:
    print('\nERROR: Factor analysis failed\n')
    print(f'{str(e)}')
    raise

### Step 2.2: Visualize Initial Portfolio Style Factor Exposures

View the top contributing style factors in your initial portfolio.

In [ ]:
fig = analytics.create_style_factor_chart(initial_analysis, rows=5, title='Initial Portfolio - Style Factor Exposures')
fig.show()

print('\nStyle factors displayed (e.g., Volatility, Market Sensitivity, Momentum, Size, Value, etc.)')

### Step 2.3: Historical Performance Metrics

View dynamic performance metrics for the initial portfolio over the past year.

In [ ]:
# Create dynamic performance chart with time series data from liquidity endpoint
# Shows cumulative PnL and normalized performance over time
performance_chart = analytics.create_dynamic_performance_chart(
    initial_analysis, title='Initial Portfolio - Historical Performance Metrics'
)
performance_chart.show()

print('\nUse the dropdown menu above to toggle between:')
print('- Cumulative PnL')
print('- Normalized Performance')

## Step 3: Define Your Optimizer Universe

An optimizer universe corresponds to the assets that can be used when constructing an optimization, which can be created
using the `OptimizerUniverse` class:

| Parameter | Description | Type| Default Value|
|-----------------|---------------|-------------|-------------
| `assets`      | Assets to include in the universe. |`List[Asset]`| N/A |
| `explode_composites`     | Explode indices, ETFs, and baskets and include their underliers in the universe. |`boolean`| `True` |
| `exclude_initial_position_set_assets`       | Exclude assets in the initial holdings from the optimization. | `boolean` | `False` |
| `exclude_corporate_actions_types`     | Set of of corporate actions to be excluded in the universe. |`List[CorporateActionsTypes]`| `[]` |
| `exclude_hard_to_borrow_assets`       | Exclude hard to borrow assets from the universe. | `boolean` | `False` |
| `exclude_restricted_assets`       | Exclude assets on restricted trading lists from the universe. | `boolean` | `False` |
| `min_market_cap`       | Lowest market cap allowed for any universe constituent. | `float` | `None` |
| `max_market_cap`       | Highest market cap allowed for any universe constituent. | `float` | `None` |

In [ ]:
asset_list = ['SPY US']
asset_list_resolved = [Asset.get(x, AssetIdentifier.BLOOMBERG_COMPOSITE_ID) for x in asset_list]

universe = OptimizerUniverse(
    assets=asset_list_resolved,
    explode_composites=True,
    exclude_initial_position_set_assets=True,
    exclude_restricted_assets=True,
    exclude_hard_to_borrow_assets=True,
    exclude_corporate_actions_types=[CorporateActionsTypes.Mergers],
)

## Step 4: Define Your Risk Model and Factor Risk Constraints

You can run the optimizer using a factor risk model of your choice, so long as you have a license for it, by leveraging
the `FactorRiskModel` class. For any factor in the risk model, you can set more granular constraints on the optimized
portfolio's exposure to the factor.

In this example, let's use the Axioma AXUS4S model and limit the final exposure to Volatility be $10,000 and
the final exposure of Market Sensitivity to be 5,000.

### Pulling Factor Names from Risk Models

You can run the code below to pull the factor names for your model. 

In [ ]:
risk_model = FactorRiskModel.get('AXIOMA_AXUS4S')

risk_model_factors = risk_model.get_many_factors(dt.date(2025, 4, 9))
risk_model_factors_names = pd.DataFrame([x.name for x in risk_model_factors])
pd.set_option('display.max_rows', 87)
print(risk_model_factors_names)

### Set your Factor Constraints

Note that different models might have different sets of factor names, if the code below doesn't work please confirm that the factor name used exists in the Risk Model as shown above. 

In [ ]:
factor_constraints = [
    FactorConstraint(risk_model.get_factor('Volatility'), 10000),
    FactorConstraint(risk_model.get_factor('Market Sensitivity'), 5000),
]

## Step 5: Define Other Optimization Constraints

Outside factor-specific constraints, it's also possible to limit the holding value of individual assets, assets
belonging to a particular GICS sector, and/or assets in a particular country of domicile in the optimization.

In this example, let's constrain the optimization to have 0-50% Microsoft and limit the optimization's notional
coming from Energy and Health Care assets to each be 0-80%. We also limit exposure from gics industry (one level more granular than sector) to be from [0, 50%]

In [ ]:
asset_constraints = [
    AssetConstraint(Asset.get('MSFT UW', AssetIdentifier.BLOOMBERG_ID), 0, 50),
]

sector_constraints = [SectorConstraint('Energy', 0, 80), SectorConstraint('Health Care', 0, 80)]
industry_constraints = [IndustryConstraint('Energy Equipment & Services', 0, 50)]
country_constraints = [CountryConstraint('United States', 0, 100)]

## Step 6: Configure Your Optimization Settings

All other settings for the optimization can be set via the `OptimizerSettings` class:

| Parameter          | Description | Type| Default Value|
|--------------------|---------------|-------------|-------------
| `notional`         | Max gross notional value of the optimization |`float`| `10000000` |
| `allow_long_short` | Allow a long/short optimization |`boolean`| `False` |
| `min_names`        | Minimum number of assets in the optimization |`float`| `0` |
| `max_names`        | Maximum number of assets in the optimization |`float`| `100` |
| `min_weight_per_constituent`       | Minimum weight of each constituent in the optimization |`float`| `None` |
| `max_weight_per_constituent`       | Maximum weight of each constituent in the optimization |`float`| `None` |
| `max_adv`        | Maximum average daily volume of each constituent in the optimization (in percent) |`float`| `15` |
| `constraint_priorities`        | Priorities to relax some or all constraints in the strategy |`ConstraintPriorities`| `None` |


Let's put above parameters in action. In below cell 

    * style exposure constraint is tightened up
    * sector, industry, country weights constraint can be relaxed if it is not feasiable to find a solution that meets all requirements. 
    * number of names in the hedge from [0, 100]
    * weight range for constituents from hedge from [1%, 99%]
    * max daily ADV less than 15%
    * one sided hedge as allow_long_short is turned off.
    

In [ ]:
constraint_priorities = ConstraintPriorities(
    style_factor_exposures=PrioritySetting.ZERO,
    min_sector_weights=PrioritySetting.ONE,
    max_sector_weights=PrioritySetting.ONE,
    min_industry_weights=PrioritySetting.ONE,
    max_industry_weights=PrioritySetting.ONE,
    min_country_weights=PrioritySetting.ONE,
    max_country_weights=PrioritySetting.ONE,
)

settings = OptimizerSettings(
    allow_long_short=False,
    constraint_priorities=constraint_priorities,
    min_names=0,
    max_names=100,
    min_weight_per_constituent=0.01,
    max_weight_per_constituent=0.99,
    max_adv=15,
)

## Step 7: Create and Run a Strategy

It's finally time to take all these parameters and construct an optimizer strategy using the `OptimizerStrategy` class in which all the building blocks configured from previous steps are put together. 

    * initial portfolio
    * style, sector, industry and country constraints
    * constraint priorities
    * optimization settings with number of names, weight limits and liquitdy limits on a one sided hedge
    * optimization objectives to minimize on factor risk only


In [ ]:
constraints = OptimizerConstraints(
    asset_constraints=asset_constraints,
    sector_constraints=sector_constraints,
    factor_constraints=factor_constraints,
    industry_constraints=industry_constraints,
)

# define specific risk penalty to be 0 to optimize on factor risk only
risk_term = OptimizerObjectiveTerm(params={'specific_weight': 0, 'factor_weight': 1})
objective_parameters = OptimizerObjectiveParameters(terms=[risk_term])

strategy = OptimizerStrategy(
    initial_position_set=position_set,
    constraints=constraints,
    settings=settings,
    universe=universe,
    risk_model=risk_model,
    objective=OptimizerObjective.MINIMIZE_FACTOR_RISK,
    objective_parameters=objective_parameters,
)

In [ ]:
strategy.run(optimizer_type=OptimizerType.AXIOMA_PORTFOLIO_OPTIMIZER)

# Get hedge and hedged portfolio
hedge = strategy.get_optimization()  # Returns just the optimization results as a PositionSet object
hedged_portfolio = strategy.get_optimized_position_set()

print(f'\nHedge Portfolio: {len(hedge.positions)} positions')
hedge_df = pd.DataFrame(
    [
        {
            'Asset': p.identifier,
            'Quantity': f'{p.quantity:.0f}' if p.quantity else '',
            'Weight': f'{p.weight:.2%}' if p.weight else '',
        }
        for p in hedge.positions
    ]
)
display(hedge_df)

### Step 7.1: Analyze Hedged Portfolio Performance

View the cumulative PnL performance of the hedged portfolio.

In [ ]:
performance_df = strategy.get_cumulative_pnl_performance(HedgeTarget.HEDGED_TARGET)

if not performance_df.empty:
    print('\nHedged Portfolio Cumulative PnL Performance:')
    fig = analytics.create_performance_chart(
        performance_df, metric='cumulativePnl', title='Hedged Portfolio - Cumulative PnL'
    )
    fig.show()
else:
    print('No performance data available')

### Step 7.2: Visualize Hedged Portfolio Style Factor Exposures

View the style factor exposures after hedging.

In [ ]:
# Create style factor chart (excludes GICS classification data)
hedged_factor_exposures = strategy.get_style_factor_exposures(HedgeTarget.HEDGED_TARGET)
hedged_analysis = analytics.convert_hedge_factor_exposures(hedged_factor_exposures)
fig = analytics.create_style_factor_chart(hedged_analysis, rows=5, title='Hedged Portfolio - Style Factor Exposures')
fig.show()

print('\nStyle factors displayed (e.g., Volatility, Market Sensitivity, Momentum, Size, Value, etc.)')

### Step 7.3: Compare Initial vs Hedged Portfolio

Compare key risk metrics between the initial and hedged portfolios.

In [ ]:
# Display performance summary from strategy in separate tables
print('\nPerformance Summary from Hedge Calculation:')
try:
    summary = strategy.get_performance_summary()

    # Optionally display all metrics in one combined table
    print('\n--- All Metrics (Combined) ---')
    display(summary['combined'])

except Exception as e:
    print(f'Could not retrieve performance summary: {str(e)}')

#### Side-by-Side Factor Comparison

Compare all style factor exposures between initial and hedged portfolios using a grouped bar chart.

In [ ]:
# Create side-by-side comparison of all style factors
print('Creating Style Factor Comparison Chart...\n')
comparison_chart = analytics.create_factor_heatmap_comparison(
    initial_analysis, hedged_analysis, title='Style Factor Comparison: Initial vs Hedged Target Portfolio'
)
comparison_chart.show()

print('\nThis chart displays all style factors with side-by-side bars for easy comparison.')
print('Blue bars = Initial Portfolio | Green bars = Hedged Target Portfolio')
print('Factors are sorted by initial portfolio exposure (largest to smallest by absolute value).')
print('A vertical line at zero helps identify positive vs negative exposures.')

#### Note:
If code above fails to find optimization, relax the constraints and try again.

## Step 8: Create a Basket or Portfolio With Your Results

Now that you have a position set for your optimization and your optimized position set, you can upload either to a basket or
portfolio by following the following tutorials:

- [Create a Basket](https://nbviewer.org/github/goldmansachs/gs-quant/blob/master/gs_quant/documentation/06_baskets/tutorials/Basket%20Create.ipynb)
- [Create a Portfolio](https://nbviewer.org/github/goldmansachs/gs-quant/blob/master/gs_quant/documentation/10_one_delta/scripts/portfolios/01_Create%20New%20Historical%20Portfolio.ipynb)

*Other questions? Reach out to the [Portfolio Analytics team](mailto:gs-marquee-analytics-support@gs.com)!*
